# Capstone Project - The Battle of Neighborhoods

## Import Libraries

### In this section we import the libraries that will be required to process the data.

The first library is Pandas. Pandas is an open source, BSD-licensed library, providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


## Download and Explore Dataset

Download and Explore Dataset Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


### Tranform the data into a pandas dataframe

In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
neighborhoods['Borough'].value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods superimposed on top.

In [8]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [9]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Borough'], Brooklyn_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues

In [10]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [11]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'H4UYJU3CMIE02IFAHGV3I5DXFUNIQKW3D4QWZLWKGB45EWBT'
CLIENT_SECRET = 'Z4KBLX4LIM3BMHWKU5WBKMGX50TBDIAIXRPMEI20H4FEKYSB'
VERSION = '20181020'

In [12]:
#https://developer.foursquare.com/docs/resources/categories
#Indian = 4bf58dd8d48988d10f941735


neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
newyork_venues_indian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d10f941735')
newyork_venues_indian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Bombay Grill,40.622331,-74.031561,Indian Restaurant
1,Bay Ridge,40.625801,-74.030621,Mohims Indian Cuisine,40.632732,-74.027260,Indian Restaurant
2,Bay Ridge,40.625801,-74.030621,King of Tandoor,40.631836,-74.032477,Indian Restaurant
3,Bay Ridge,40.625801,-74.030621,India Passage,40.631856,-74.027576,Indian Restaurant
4,Bay Ridge,40.625801,-74.030621,Taj Mahal,40.632373,-74.027193,Indian Restaurant


In [13]:
newyork_venues_indian.shape

(272, 7)

In [14]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [15]:
map_newyork_indian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_indian, 'red', map_newyork_indian)

map_newyork_indian

In [16]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [18]:
Brooklyn_grouped = newyork_venues_indian.groupby('Neighborhood').count()
Brooklyn_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,1,1,1,1,1,1
Bay Ridge,5,5,5,5,5,5
Bedford Stuyvesant,6,6,6,6,6,6
Bensonhurst,1,1,1,1,1,1
Boerum Hill,11,11,11,11,11,11
Borough Park,1,1,1,1,1,1
Brighton Beach,2,2,2,2,2,2
Broadway Junction,1,1,1,1,1,1
Brooklyn Heights,5,5,5,5,5,5


### Analyze Each Neighborhood

In [19]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(newyork_venues_indian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = newyork_venues_indian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Neighborhood,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Food Truck,Halal Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,North Indian Restaurant
0,Bay Ridge,0,0,0,0,0,0,0,1,0,0
1,Bay Ridge,0,0,0,0,0,0,0,1,0,0
2,Bay Ridge,0,0,0,0,0,0,0,1,0,0
3,Bay Ridge,0,0,0,0,0,0,0,1,0,0
4,Bay Ridge,0,0,0,0,0,0,0,1,0,0


In [20]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped

,Neighborhood,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Food Truck,Halal Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,North Indian Restaurant
0,Bath Beach,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
1,Bay Ridge,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
2,Bedford Stuyvesant,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
3,Bensonhurst,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
4,Boerum Hill,0.000000,0.000000,0.090909,0.0,0.000000,0.0000,0.0,0.909091,0.000000,0.000000
5,Borough Park,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
6,Brighton Beach,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
7,Broadway Junction,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000
8,Brooklyn Heights,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.800000,0.200000,0.000000
9,Bushwick,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,1.000000,0.000000,0.000000


In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
1,Bay Ridge,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
2,Bedford Stuyvesant,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
3,Bensonhurst,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
4,Boerum Hill,Indian Restaurant,Chinese Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Caribbean Restaurant,Café


### Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = 5

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 2, 0, 0, 0, 3, 4,
       4, 0, 2, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 3, 4, 1, 3, 0, 0, 0,
       2, 0, 0, 0, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



Brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
2,Brooklyn,Sunset Park,40.645103,-74.010316,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
3,Brooklyn,Greenpoint,40.730201,-73.954241,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
Brooklyn_merged=Brooklyn_merged.dropna()
Brooklyn_merged.isna().sum()

Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [26]:
Brooklyn_merged=Brooklyn_merged[Brooklyn_merged['Neighborhood']!='Gravesend']
Brooklyn_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
2,Brooklyn,Sunset Park,40.645103,-74.010316,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
3,Brooklyn,Greenpoint,40.730201,-73.954241,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
5,Brooklyn,Brighton Beach,40.576825,-73.965094,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
8,Brooklyn,Flatbush,40.636326,-73.958401,4.0,Indian Restaurant,Halal Restaurant,Café,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant
9,Brooklyn,Crown Heights,40.670829,-73.943291,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
10,Brooklyn,East Flatbush,40.641718,-73.936103,0.0,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
1,Bensonhurst,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
2,Sunset Park,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
3,Greenpoint,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
5,Brighton Beach,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
6,Sheepshead Bay,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
7,Manhattan Terrace,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
9,Crown Heights,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
10,East Flatbush,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
12,Windsor Terrace,Indian Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café


In [29]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Remsen Village,Caribbean Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Café


In [30]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Brooklyn Heights,Indian Restaurant,Indian Sweet Shop,North Indian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
61,Fulton Ferry,Indian Restaurant,Food Truck,Indian Sweet Shop,North Indian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
62,Vinegar Hill,Indian Sweet Shop,Indian Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
65,Dumbo,Indian Restaurant,Indian Sweet Shop,North Indian Restaurant,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café


In [31]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Prospect Lefferts Gardens,Indian Restaurant,Coffee Shop,Caribbean Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Chinese Restaurant,Café
56,Rugby,Indian Restaurant,Caribbean Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Café
69,Erasmus,Indian Restaurant,Caribbean Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Café


In [32]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Indian Restaurant,Halal Restaurant,Café,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant
11,Kensington,Indian Restaurant,Indian Chinese Restaurant,North Indian Restaurant,Indian Sweet Shop,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
23,Fort Greene,Indian Restaurant,Chinese Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Caribbean Restaurant,Café
40,Downtown,Indian Restaurant,Chinese Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Caribbean Restaurant,Café
41,Boerum Hill,Indian Restaurant,Chinese Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Caribbean Restaurant,Café
46,Midwood,Indian Restaurant,Halal Restaurant,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Food Truck,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Café
47,Prospect Park South,Indian Restaurant,Caribbean Restaurant,Café,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant
54,Ditmas Park,Indian Restaurant,Caribbean Restaurant,Café,North Indian Restaurant,Indian Sweet Shop,Indian Chinese Restaurant,Halal Restaurant,Food Truck,Coffee Shop,Chinese Restaurant
